# State Student Spending Case Study
#### Examining any relationship between student sucess or lack thereof with how much money the state has invested


### Necessary Steps for Data Story
##### 1) Clean the Data 
    a. Import Data
    b. Transform variables
##### 2) Exploratory Data Analysis
    a. Visualize the data
##### 3) Preprocess and Train Data
##### 4) Create Machine Learning Model
##### 5) Summarize Findings

In [1]:
#import necessary modules

import pandas as pd
import numpy as np

# 1a. Import Data

In [2]:
#import Graduation rate, Dropout rate, State Spending, Population, & State GDP data
Grad = pd.read_csv('Data/Completion1318edit.csv')
Drop = pd.read_csv('Data/Dropout1318edit.csv')
StateSpend = pd.read_excel('Data/StateSpendingStudents.xlsx')
Pop = pd.read_csv('Data/Pop10-19.csv')
GDP = pd.read_csv('Data/State Quarterly GDP 2005 - 2021 - Sheet1.csv')

In [3]:
#import tax data 
Tax13 = pd.read_csv('Data/2013 State Taxes.csv').replace('X', 0)
Tax14 = pd.read_csv('Data/2014 State Taxes.csv').replace('X', 0)
Tax15 = pd.read_csv('Data/2015 State Taxes.csv').replace('X', 0)
Tax16 = pd.read_csv('Data/2016 State Taxes.csv').replace('X', 0)
Tax17 = pd.read_csv('Data/2017 State Taxes.csv').replace('X', 0)
Tax18 = pd.read_csv('Data/2018 State Taxes.csv').replace('X', 0)
Tax19 = pd.read_csv('Data/2019 State Taxes.csv').replace('X', 0)


#Note to self to automate process:
#Try a dictionary. Start with empty dictionary. Wild cards to read itieratively * .csv

## Cleaning Tax Data

In [4]:
#Examing the column names: Need to change them to match the rest of the data
#Tax13.columns


In [5]:
#Dictionary to change state abbreviations to full state names
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'District of Columbia': 'DC',
}

In [6]:
#Load all the individual tax files into one frame with an appropriate year varibable

TaxList = [Tax13, Tax14, Tax15, Tax16, Tax17, Tax18, Tax19]

TaxFile = pd.DataFrame()


initial_year = 2013
for loc_file in TaxList:
  loc_file = loc_file.iloc[:,1:]
  loc_file['Year'] = str(initial_year)
  loc_file = loc_file.melt(id_vars='Year', value_name = 'tax_all', var_name='State')
  TaxFile  = TaxFile.append(loc_file)
  initial_year = initial_year + 1

In [7]:
#Sum all the different types of tax categories
TaxFile['tax_all'] = TaxFile['tax_all'].astype('float')

AllTaxes = TaxFile.groupby(['State', 'Year'])['tax_all'].sum().reset_index()
AllTaxes.head()


,State,Year,tax_all
0,AK,2013,5132811.0
1,AK,2014,3392870.0
2,AK,2015,1363350.0
3,AK,2016,896921.0
4,AK,2017,1208929.0


In [8]:
#Fix the naming

AllTaxes['State'] = AllTaxes['State'].replace(us_state_abbrev)
AllTaxes.head()

,State,Year,tax_all
0,Alaska,2013,5132811.0
1,Alaska,2014,3392870.0
2,Alaska,2015,1363350.0
3,Alaska,2016,896921.0
4,Alaska,2017,1208929.0


## Cleaning Median Income Data

In [9]:
#Read in Median Income (Deleted out Standard Deviation as well as repetitive information from original file)
MedianInc = pd.read_csv('Data/median_income_cleaned.csv')

#Flip
MedianInc = MedianInc.melt(id_vars = 'State', var_name='Year', value_name = 'MedInc')

MedianInc.head()
#Delete unnecessary columns and repetitive information.

,State,Year,MedInc
0,United States,2018,63179
1,Alabama,2018,49936
2,Alaska,2018,68734
3,Arizona,2018,62283
4,Arkansas,2018,49781


## Cleaning Graduation and Drop Out Data

In [10]:
#examine data
print(Grad.columns)

Index(['State', '2012-2013', '2013-2014', '2014-2015', '2015-2016',
       '2016-2017', '2017-2018'],
      dtype='object')


In [11]:
#Rename columns so they match years in other files.
oldname = Grad.columns
newname = ['State', 2013, 2014, 2015, 2016, 2017, 2018 ]
Grad = Grad.rename(columns=dict(zip(oldname, newname)))

#Flip so States are columns
Grad = Grad.melt(id_vars = ['State'], var_name='Year', value_name = 'Graduates')


#Examine
Grad.head()

,State,Year,Graduates
0,United States,2013,81.4
1,Alabama,2013,80.0
2,Alaska,2013,71.8
3,Arizona,2013,75.1
4,Arkansas,2013,84.9


In [12]:
#examine data
print(Drop.head())
print(Drop.columns)

           State 2013 2014 2015  2016 2017 2018
0  United States  6.8  6.3  6.0   5.8  5.4  5.3
1        Alabama  8.9  7.7  7.4   6.9  6.6  5.4
2         Alaska  6.2  2.8  8.8   5.6  5.3  4.4
3        Arizona  9.4  8.5  9.2   7.6  8.0  7.7
4       Arkansas  6.8  6.8  6.7   7.2  5.7  5.6
Index(['State', '2013', '2014', '2015', '2016', '2017', '2018'], dtype='object')


In [13]:
#Flip so States are columns
Drop = Drop.melt(id_vars = ['State'], var_name='Year', value_name = 'Dropout')

#Examine
Drop.head()

,State,Year,Dropout
0,United States,2013,6.8
1,Alabama,2013,8.9
2,Alaska,2013,6.2
3,Arizona,2013,9.4
4,Arkansas,2013,6.8


## Clean State Spending Data

In [14]:
#StateSpend.head() to examine data before cleaning

In [15]:
#Realign with States as columns
StateSpend.rename(columns = {'state': 'State'}, inplace = True)
StateSpend = StateSpend.melt(id_vars = ['State'], var_name='Year', value_name = 'StateSpend')
StateSpend.head()

#Examine
print(StateSpend.head())

        State  Year  StateSpend
0     Alabama  1997   3271969.0
1      Alaska  1997   1042311.0
2     Arizona  1997   3388165.0
3    Arkansas  1997   1960613.0
4  California  1997  28708364.0


## Clean Population Data

In [16]:
#Examine the Population
# print(Pop.head())

In [17]:
#Fix geographic area names & delete unnecessary strings

Pop = Pop.drop(columns = ['Census', 'Estimates Base']).iloc[:58,]
Pop['Geographic Area'] = Pop['Geographic Area'].str.replace('.','')


# Combine and examine
Pop = Pop.melt(id_vars = 'Geographic Area',var_name='Year',value_name = 'Population')
Pop.tail()

,Geographic Area,Year,Population
555,Virginia,2019,"8,535,519"
556,Washington,2019,"7,614,893"
557,West Virginia,2019,"1,792,147"
558,Wisconsin,2019,"5,822,434"
559,Wyoming,2019,"578,759"


In [18]:
#Change column name to match other datasets
Pop = Pop.rename(columns={'Geographic Area' : 'State'})
Pop.head()

,State,Year,Population
0,United States,2010,"309,321,666"
1,Northeast,2010,"55,380,134"
2,Midwest,2010,"66,974,416"
3,South,2010,"114,866,680"
4,West,2010,"72,100,436"


## Clean GDP  Data

In [19]:
#Examine GDP data before edits
# GDP.head()

In [20]:
# In order to average the GDP by year we need to extract the year
GDP['Year'] = GDP['DATE'].str[:4]
GDP.drop(columns = 'DATE', inplace = True)
GDP.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Year
0,153077.8,37393.9,219879.5,87648.2,1071327.2,215960.0,205563.8,50938.8,84367.6,679535.2,...,224177.7,959728.9,90791.3,23305.0,351558.7,289799.4,51590.1,222355.1,25403.1,2005
1,155409.9,38641.2,225374.3,88484.4,1147944.3,218742.0,207535.4,49992.1,85216.8,688958.5,...,225712.9,966652.1,92408.8,23288.1,358082.4,294263.7,52859.3,224122.0,26571.8,2005
2,156809.7,40259.9,232290.1,89890.7,1247734.4,222965.7,208749.0,50351.7,85499.3,708198.9,...,229952.8,993026.3,95362.0,23560.3,365491.2,295654.5,53610.3,228348.1,27453.0,2005
3,160315.8,42859.7,235386.5,92549.3,1366166.5,223561.9,212795.6,56830.1,86313.2,712347.7,...,232661.9,1025403.5,98650.0,23850.2,368236.9,303717.2,55111.1,227537.6,29448.8,2005
4,161732.8,42664.7,242418.5,92836.0,1387551.8,228594.0,221184.4,54220.3,87757.7,732154.4,...,236210.0,1054437.0,102069.5,24289.8,375995.3,310466.5,55639.7,232585.7,30455.1,2006


In [21]:
#Melt it down 
GDP = GDP.melt(id_vars = ['Year'], var_name='State', value_name = 'GDP')
GDP.head()

,Year,State,GDP
0,2005,Alabama,153077.8
1,2005,Alabama,155409.9
2,2005,Alabama,156809.7
3,2005,Alabama,160315.8
4,2006,Alabama,161732.8


In [22]:
#Find the average yearly GDP
YearlyGDP = GDP.groupby(['State', 'Year'])['GDP'].mean().reset_index()

In [23]:
YearlyGDP.head()

,State,Year,GDP
0,Alabama,2005,156403.30
1,Alabama,2006,164347.85
2,Alabama,2007,170263.85
3,Alabama,2008,172677.25
4,Alabama,2009,169128.50


## Clean Unemployment Rates

In [24]:
# Empty data frame
Unemployment = pd.DataFrame()


for i in np.arange(2009,2018,1):
    print("Reading: {}".format(str(i)))
    
    year_data = pd.read_excel('Data/UnemploymentRateData_Parsed.xlsx', sheet_name = str(i))
    melt_data = pd.melt(year_data, id_vars = 'State', var_name = ['Month'], value_name = 'value')

    aggregated_data = melt_data.groupby('State').agg({'value' : ['min', 'max', 'mean']}).reset_index()
    aggregated_data.columns = ['State', 'min_unemploymentRate', 'max_unemploymentRate', 'mean_unemploymentRate']
    aggregated_data['Year'] = str(i)
    
# Logic to add each years data to the end of previously created data
Unemployment = Unemployment.append(aggregated_data)

Unemployment.head()

Reading: 2009
Reading: 2010
Reading: 2011
Reading: 2012
Reading: 2013
Reading: 2014
Reading: 2015
Reading: 2016
Reading: 2017


,State,min_unemploymentRate,max_unemploymentRate,mean_unemploymentRate,Year
0,Alabama,3.8,5.5,4.408333,2017
1,Alaska,7.0,7.2,7.141667,2017
2,Arizona,4.7,5.2,4.858333,2017
3,Arkansas,3.6,3.7,3.666667,2017
4,California,4.5,5.2,4.775000,2017


## Merge ALL the clean files

In [25]:
MedianInc['Year']

0       2018
1       2018
2       2018
3       2018
4       2018
        ... 
1191    1995
1192    1995
1193    1995
1194    1995
1195    1995
Name: Year, Length: 1196, dtype: object

In [26]:
#Change them all to strings because??

Grad['Year'] = Grad['Year'].astype('str')
Grad['State'] = Grad['State'].astype('str')

Drop['Year'] = Drop['Year'].astype('str')
Drop['State'] = Drop['State'].astype('str')

StateSpend['Year'] = StateSpend['Year'].astype('str')
StateSpend['State'] = StateSpend['State'].astype('str')

#Pop['Population'] = Pop['Population'].astype('Float64')

MedianInc['Year'] = MedianInc['Year'].astype('str')
MedianInc['State'] = MedianInc['State'].astype('str')
#MedianInc['MedInc'] = MedianInc['MedInc'].astype('Float64')

In [27]:
#merge graduate and drop data
grad_drop_merge = Grad.merge(Drop, on = ['Year', 'State'], how = 'left')
grad_drop_merge.head()

,State,Year,Graduates,Dropout
0,United States,2013,81.4,6.8
1,Alabama,2013,80.0,8.9
2,Alaska,2013,71.8,6.2
3,Arizona,2013,75.1,9.4
4,Arkansas,2013,84.9,6.8


In [28]:
#Merge state spending data
all_spend_merge = grad_drop_merge.merge(StateSpend, on = ['Year', 'State'], how = 'left')
all_spend_merge.head()

,State,Year,Graduates,Dropout,StateSpend
0,United States,2013,81.4,6.8,NaN
1,Alabama,2013,80.0,8.9,6461916.0
2,Alaska,2013,71.8,6.2,2202551.0
3,Arizona,2013,75.1,9.4,6675200.0
4,Arkansas,2013,84.9,6.8,4222408.0


In [29]:
# Merge in population
all_pop_merge = all_spend_merge.merge(Pop, on = ['Year', 'State'], how = 'left')
all_pop_merge.head()

,State,Year,Graduates,Dropout,StateSpend,Population
0,United States,2013,81.4,6.8,NaN,"315,993,715"
1,Alabama,2013,80.0,8.9,6461916.0,"4,830,081"
2,Alaska,2013,71.8,6.2,2202551.0,"737,068"
3,Arizona,2013,75.1,9.4,6675200.0,"6,632,764"
4,Arkansas,2013,84.9,6.8,4222408.0,"2,959,400"


In [30]:
#Merge in Taxes
all_tax_merge = all_pop_merge.merge(AllTaxes, on = ['Year', 'State'], how = 'left')
all_tax_merge.head()

,State,Year,Graduates,Dropout,StateSpend,Population,tax_all
0,United States,2013,81.4,6.8,NaN,"315,993,715",NaN
1,Alabama,2013,80.0,8.9,6461916.0,"4,830,081",9270919.0
2,Alaska,2013,71.8,6.2,2202551.0,"737,068",5132811.0
3,Arizona,2013,75.1,9.4,6675200.0,"6,632,764",13796918.0
4,Arkansas,2013,84.9,6.8,4222408.0,"2,959,400",8586767.0


In [31]:
#Merge in Median Income
all_median_merge = all_tax_merge.merge(MedianInc, on = ['Year', 'State'], how = 'left')
all_median_merge.head()

,State,Year,Graduates,Dropout,StateSpend,Population,tax_all,MedInc
0,United States,2013,81.4,6.8,NaN,"315,993,715",NaN,NaN
1,Alabama,2013,80.0,8.9,6461916.0,"4,830,081",9270919.0,NaN
2,Alaska,2013,71.8,6.2,2202551.0,"737,068",5132811.0,NaN
3,Arizona,2013,75.1,9.4,6675200.0,"6,632,764",13796918.0,NaN
4,Arkansas,2013,84.9,6.8,4222408.0,"2,959,400",8586767.0,NaN


In [32]:
#Merge in GDP
all_gdp_merge = all_median_merge.merge(YearlyGDP, on = ['Year', 'State'], how = 'left')
all_gdp_merge.head()

,State,Year,Graduates,Dropout,StateSpend,Population,tax_all,MedInc,GDP
0,United States,2013,81.4,6.8,NaN,"315,993,715",NaN,NaN,NaN
1,Alabama,2013,80.0,8.9,6461916.0,"4,830,081",9270919.0,NaN,192166.525
2,Alaska,2013,71.8,6.2,2202551.0,"737,068",5132811.0,NaN,56625.250
3,Arizona,2013,75.1,9.4,6675200.0,"6,632,764",13796918.0,NaN,275007.525
4,Arkansas,2013,84.9,6.8,4222408.0,"2,959,400",8586767.0,NaN,112764.575


In [33]:
#Merge in Unemployment
all_merge = all_gdp_merge.merge(Unemployment, on = ['Year', 'State'], how = 'left')
all_merge.head()

,State,Year,Graduates,Dropout,StateSpend,Population,tax_all,MedInc,GDP,min_unemploymentRate,max_unemploymentRate,mean_unemploymentRate
0,United States,2013,81.4,6.8,NaN,"315,993,715",NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2013,80.0,8.9,6461916.0,"4,830,081",9270919.0,NaN,192166.525,NaN,NaN,NaN
2,Alaska,2013,71.8,6.2,2202551.0,"737,068",5132811.0,NaN,56625.250,NaN,NaN,NaN
3,Arizona,2013,75.1,9.4,6675200.0,"6,632,764",13796918.0,NaN,275007.525,NaN,NaN,NaN
4,Arkansas,2013,84.9,6.8,4222408.0,"2,959,400",8586767.0,NaN,112764.575,NaN,NaN,NaN


# Begin Exploratory Data Analysis

In [34]:
#Limit to years of interest
#StateSpendTrain = StateSpend.iloc[16:20]

#State Spend Documentation ends after 2016
#StateSpendTest = StateSpend.iloc[19:]

#print(StateSpendTrain)
#print(StateSpendTest)